In [1]:
from policy_gradient import PolicyGradient
import matplotlib.pyplot as plt

In [2]:
import random
import numpy as np
from statistics import median, mean
from collections import Counter

In [3]:
from IPython.display import clear_output # only for jupyter notebook
from time import sleep
import pickle

In [4]:
# import game env
from puzzle import GameGrid
env = GameGrid()
env.reset()

[[0, 0, 0, 0], [0, 0, 0, 0], [0, 2, 0, 0], [0, 0, 0, 2]]

In [6]:
load_path = None #"output/weights/CartPole-v0.ckpt"
save_path = "output/weights/experiment3.ckpt"

PG = PolicyGradient(
    n_x = 64,
    n_y = 4,
    learning_rate=LR,
    reward_decay=0.95,
    load_path=load_path,
    save_path=save_path
)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [7]:
def convertTo1D(matrix):
    return np.array(np.array(matrix).flatten().tolist())

In [15]:
def getGradientMatrix(matrix):
    if isinstance(matrix, list):
        matrix = np.matrix(matrix)
    m, n = matrix.shape
    M1 = []
    #left - right
    for i in range(m):
        M1.append([])
        for j in range(n - 1):
            M1[i].append(matrix[i, j + 1] - matrix[i, j])
    
    M2 = []
    #right - left
    for i in range(m):
        M2.append([])
        for j in range(n - 1):
            M2[i].append(matrix[i, n - 1 - (j + 1)] - matrix[i, n - 1 - j])
    
    M3 = []
    #top - bottom
    for i in range(m - 1):
        M3.append([])
        for j in range(n):
            M3[i].append(matrix[i + 1, j] - matrix[i, j])

    M4 = []
    #top - bottom
    for i in range(m - 1):
        M4.append([])
        for j in range(n):
            M4[i].append(matrix[m - 1 - (i + 1), j] - matrix[m - 1 - i, j])
    
    return M1, M2, M3, M4

In [17]:
def ModifyState(matrix):
    M0 = convertTo1D(matrix)
    M1, M2, M3, M4 = getGradientMatrix(matrix)
    M = [M0, M1, M2, M3, M4]
    result = []
    for m in M:
        for i in np.array(m).flatten().tolist():
            result.append(i)
    result = np.array(result)
    return result

In [33]:
RENDER_ENV = False
EPISODES = 10000
rewards = []
max_episode_values = []
RENDER_REWARD_MIN = 50
LR = 0.01

for episode in range(EPISODES):
    obs = env.reset()
    observation = ModifyState(obs)
    episode_reward = 0
    episode_max_value = 0
    
    while True:
        if RENDER_ENV:
            clear_output()
            env.display_state()
            sleep(0.5)
            
        # 1. Choose an action based on observation
        action = PG.choose_action(observation)
        
        # 2. Take action in the environment
        obs_, reward, done, info = env.step(action.item(), 0.8)
        obs_ = ModifyState(obs_)
        
        # check if it is unfeasible move
        if np.array_equal(observation, obs_):
            # use another random move
            action = random.randrange(0,4)
            obs_, reward, done, info = env.step(action)
            obs_ = ModifyState(obs_)
            
        # 3. Store transition for training
        PG.store_transition(observation, action, reward)
        
        # 4. Save new observation
        observation = obs_
        
        # 5. Save max value in the game
        if episode_max_value < env.highest_score():
            episode_max_value = env.highest_score()
        
        if done:
            episode_rewards_sum = sum(PG.episode_rewards)
            rewards.append(episode_rewards_sum)
            max_reward_so_far = np.amax(rewards)
            
            max_episode_values.append(episode_max_value)

            #print("==========================================")
            print("Episode: ", episode)
            #print("Reward: ", episode_rewards_sum)
            #print("Max reward so far: ", max_reward_so_far)

            # 4. Train neural network
            discounted_episode_rewards_norm = PG.learn()

            # Render env if we get to rewards minimum
            #if max_reward_so_far > RENDER_REWARD_MIN: RENDER_ENV = True
            break

Episode:  0
Episode:  1
Episode:  2
Episode:  3
Episode:  4
Episode:  5
Episode:  6
Episode:  7
Episode:  8
Episode:  9
Episode:  10
Episode:  11
Episode:  12
Episode:  13
Episode:  14
Episode:  15
Episode:  16
Episode:  17
Episode:  18
Episode:  19
Episode:  20
Episode:  21
Episode:  22
Episode:  23
Episode:  24
Episode:  25
Episode:  26
Episode:  27
Episode:  28
Episode:  29
Episode:  30
Episode:  31
Episode:  32
Episode:  33
Episode:  34
Episode:  35
Episode:  36
Episode:  37
Episode:  38
Episode:  39
Episode:  40
Episode:  41
Episode:  42
Episode:  43
Episode:  44
Episode:  45
Episode:  46
Episode:  47
Episode:  48
Episode:  49
Episode:  50
Episode:  51
Episode:  52
Episode:  53
Episode:  54
Episode:  55
Episode:  56
Episode:  57
Episode:  58
Episode:  59
Episode:  60
Episode:  61
Episode:  62
Episode:  63
Episode:  64
Episode:  65
Episode:  66
Episode:  67
Episode:  68
Episode:  69
Episode:  70
Episode:  71
Episode:  72
Episode:  73
Episode:  74
Episode:  75
Episode:  76
Episode: 

Episode:  595
Episode:  596
Episode:  597
Episode:  598
Episode:  599
Episode:  600
Episode:  601
Episode:  602
Episode:  603
Episode:  604
Episode:  605
Episode:  606
Episode:  607
Episode:  608
Episode:  609
Episode:  610
Episode:  611
Episode:  612
Episode:  613
Episode:  614
Episode:  615
Episode:  616
Episode:  617
Episode:  618
Episode:  619
Episode:  620
Episode:  621
Episode:  622
Episode:  623
Episode:  624
Episode:  625
Episode:  626
Episode:  627
Episode:  628
Episode:  629
Episode:  630
Episode:  631
Episode:  632
Episode:  633
Episode:  634
Episode:  635
Episode:  636
Episode:  637
Episode:  638
Episode:  639
Episode:  640
Episode:  641
Episode:  642
Episode:  643
Episode:  644
Episode:  645
Episode:  646
Episode:  647
Episode:  648
Episode:  649
Episode:  650
Episode:  651
Episode:  652
Episode:  653
Episode:  654
Episode:  655
Episode:  656
Episode:  657
Episode:  658
Episode:  659
Episode:  660
Episode:  661
Episode:  662
Episode:  663
Episode:  664
Episode:  665
Episod

Episode:  1169
Episode:  1170
Episode:  1171
Episode:  1172
Episode:  1173
Episode:  1174
Episode:  1175
Episode:  1176
Episode:  1177
Episode:  1178
Episode:  1179
Episode:  1180
Episode:  1181
Episode:  1182
Episode:  1183
Episode:  1184
Episode:  1185
Episode:  1186
Episode:  1187
Episode:  1188
Episode:  1189
Episode:  1190
Episode:  1191
Episode:  1192
Episode:  1193
Episode:  1194
Episode:  1195
Episode:  1196
Episode:  1197
Episode:  1198
Episode:  1199
Episode:  1200
Episode:  1201
Episode:  1202
Episode:  1203
Episode:  1204
Episode:  1205
Episode:  1206
Episode:  1207
Episode:  1208
Episode:  1209
Episode:  1210
Episode:  1211
Episode:  1212
Episode:  1213
Episode:  1214
Episode:  1215
Episode:  1216
Episode:  1217
Episode:  1218
Episode:  1219
Episode:  1220
Episode:  1221
Episode:  1222
Episode:  1223
Episode:  1224
Episode:  1225
Episode:  1226
Episode:  1227
Episode:  1228
Episode:  1229
Episode:  1230
Episode:  1231
Episode:  1232
Episode:  1233
Episode:  1234
Episode:  

Episode:  1717
Episode:  1718
Episode:  1719
Episode:  1720
Episode:  1721
Episode:  1722
Episode:  1723
Episode:  1724
Episode:  1725
Episode:  1726
Episode:  1727
Episode:  1728
Episode:  1729
Episode:  1730
Episode:  1731
Episode:  1732
Episode:  1733
Episode:  1734
Episode:  1735
Episode:  1736
Episode:  1737
Episode:  1738
Episode:  1739
Episode:  1740
Episode:  1741
Episode:  1742
Episode:  1743
Episode:  1744
Episode:  1745
Episode:  1746
Episode:  1747
Episode:  1748
Episode:  1749
Episode:  1750
Episode:  1751
Episode:  1752
Episode:  1753
Episode:  1754
Episode:  1755
Episode:  1756
Episode:  1757
Episode:  1758
Episode:  1759
Episode:  1760
Episode:  1761
Episode:  1762
Episode:  1763
Episode:  1764
Episode:  1765
Episode:  1766
Episode:  1767
Episode:  1768
Episode:  1769
Episode:  1770
Episode:  1771
Episode:  1772
Episode:  1773
Episode:  1774
Episode:  1775
Episode:  1776
Episode:  1777
Episode:  1778
Episode:  1779
Episode:  1780
Episode:  1781
Episode:  1782
Episode:  

Episode:  2264
Episode:  2265
Episode:  2266
Episode:  2267
Episode:  2268
Episode:  2269
Episode:  2270
Episode:  2271
Episode:  2272
Episode:  2273
Episode:  2274
Episode:  2275
Episode:  2276
Episode:  2277
Episode:  2278
Episode:  2279
Episode:  2280
Episode:  2281
Episode:  2282
Episode:  2283
Episode:  2284
Episode:  2285
Episode:  2286
Episode:  2287
Episode:  2288
Episode:  2289
Episode:  2290
Episode:  2291
Episode:  2292
Episode:  2293
Episode:  2294
Episode:  2295
Episode:  2296
Episode:  2297
Episode:  2298
Episode:  2299
Episode:  2300
Episode:  2301
Episode:  2302
Episode:  2303
Episode:  2304
Episode:  2305
Episode:  2306
Episode:  2307
Episode:  2308
Episode:  2309
Episode:  2310
Episode:  2311
Episode:  2312
Episode:  2313
Episode:  2314
Episode:  2315
Episode:  2316
Episode:  2317
Episode:  2318
Episode:  2319
Episode:  2320
Episode:  2321
Episode:  2322
Episode:  2323
Episode:  2324
Episode:  2325
Episode:  2326
Episode:  2327
Episode:  2328
Episode:  2329
Episode:  

Episode:  2812
Episode:  2813
Episode:  2814
Episode:  2815
Episode:  2816
Episode:  2817
Episode:  2818
Episode:  2819
Episode:  2820
Episode:  2821
Episode:  2822
Episode:  2823
Episode:  2824
Episode:  2825
Episode:  2826
Episode:  2827
Episode:  2828
Episode:  2829
Episode:  2830
Episode:  2831
Episode:  2832
Episode:  2833
Episode:  2834
Episode:  2835
Episode:  2836
Episode:  2837
Episode:  2838
Episode:  2839
Episode:  2840
Episode:  2841
Episode:  2842
Episode:  2843
Episode:  2844
Episode:  2845
Episode:  2846
Episode:  2847
Episode:  2848
Episode:  2849
Episode:  2850
Episode:  2851
Episode:  2852
Episode:  2853
Episode:  2854
Episode:  2855
Episode:  2856
Episode:  2857
Episode:  2858
Episode:  2859
Episode:  2860
Episode:  2861
Episode:  2862
Episode:  2863
Episode:  2864
Episode:  2865
Episode:  2866
Episode:  2867
Episode:  2868
Episode:  2869
Episode:  2870
Episode:  2871
Episode:  2872
Episode:  2873
Episode:  2874
Episode:  2875
Episode:  2876
Episode:  2877
Episode:  

Episode:  3359
Episode:  3360
Episode:  3361
Episode:  3362
Episode:  3363
Episode:  3364
Episode:  3365
Episode:  3366
Episode:  3367
Episode:  3368
Episode:  3369
Episode:  3370
Episode:  3371
Episode:  3372
Episode:  3373
Episode:  3374
Episode:  3375
Episode:  3376
Episode:  3377
Episode:  3378
Episode:  3379
Episode:  3380
Episode:  3381
Episode:  3382
Episode:  3383
Episode:  3384
Episode:  3385
Episode:  3386
Episode:  3387
Episode:  3388
Episode:  3389
Episode:  3390
Episode:  3391
Episode:  3392
Episode:  3393
Episode:  3394
Episode:  3395
Episode:  3396
Episode:  3397
Episode:  3398
Episode:  3399
Episode:  3400
Episode:  3401
Episode:  3402
Episode:  3403
Episode:  3404
Episode:  3405
Episode:  3406
Episode:  3407
Episode:  3408
Episode:  3409
Episode:  3410
Episode:  3411
Episode:  3412
Episode:  3413
Episode:  3414
Episode:  3415
Episode:  3416
Episode:  3417
Episode:  3418
Episode:  3419
Episode:  3420
Episode:  3421
Episode:  3422
Episode:  3423
Episode:  3424
Episode:  

Episode:  3906
Episode:  3907
Episode:  3908
Episode:  3909
Episode:  3910
Episode:  3911
Episode:  3912
Episode:  3913
Episode:  3914
Episode:  3915
Episode:  3916
Episode:  3917
Episode:  3918
Episode:  3919
Episode:  3920
Episode:  3921
Episode:  3922
Episode:  3923
Episode:  3924
Episode:  3925
Episode:  3926
Episode:  3927
Episode:  3928
Episode:  3929
Episode:  3930
Episode:  3931
Episode:  3932
Episode:  3933
Episode:  3934
Episode:  3935
Episode:  3936
Episode:  3937
Episode:  3938
Episode:  3939
Episode:  3940
Episode:  3941
Episode:  3942
Episode:  3943
Episode:  3944
Episode:  3945
Episode:  3946
Episode:  3947
Episode:  3948
Episode:  3949
Episode:  3950
Episode:  3951
Episode:  3952
Episode:  3953
Episode:  3954
Episode:  3955
Episode:  3956
Episode:  3957
Episode:  3958
Episode:  3959
Episode:  3960
Episode:  3961
Episode:  3962
Episode:  3963
Episode:  3964
Episode:  3965
Episode:  3966
Episode:  3967
Episode:  3968
Episode:  3969
Episode:  3970
Episode:  3971
Episode:  

KeyboardInterrupt: 

In [ ]:
plt.plot(rewards, linewidth=0.2)
plt.xlabel("Episodes")
plt.ylabel("Rewards")
plt.gcf().subplots_adjust(left=0.15)
plt.savefig("output/graphs/training rewards.png", dpi=900)

In [ ]:
plt.plot(max_episode_values, linewidth=0.2)
plt.xlabel("Episodes")
plt.ylabel("Max. Game Values")
plt.savefig("output/graphs/training max game values.png", dpi=900)

In [ ]:
# Save the reward lists for future use
with open("output/reward values/rewards.txt", "wb") as fp:   #Pickling
    pickle.dump(rewards, fp)

with open("output/reward values/max game values.txt", "wb") as fp:   #Pickling
    pickle.dump(max_episode_values, fp)

In [32]:
# Test the policy network
num_games = 1000
rewards = []
choices = []
accepted_scores = []
RENDER_ENV = False

for each_game in range(num_games):
    score = 0
    game_memory = []
    prev_obs = []
    env.reset()
    while True:
        if RENDER_ENV:
            sleep(0.5)
            clear_output()
            env.display_state()
        #env.render()
        if len(prev_obs)==0:
            action = random.randrange(0,4)
        else:
            action = PG.choose_action(prev_obs).item()
                
        choices.append(action)
        new_observation, reward, done, info = env.step(action)
        new_observation = ModifyState(new_observation)

        # check if it is unfeasible move
        if np.array_equal(new_observation, prev_obs):
            # use another random move
            action = random.randrange(0,4)
            new_observation, reward, done, info = env.step(action)
            new_observation = ModifyState(new_observation)
        
        prev_obs = new_observation
        game_memory.append([new_observation, action])
        
        score+=reward
        if done: break
        
    if each_game%100 == 0: print(each_game)
    rewards.append(score)
    accepted_scores.append(env.highest_score())

print('Average Score:',sum(rewards)/len(rewards))
print('choice 0:{} choice 1:{} choice 2:{} choice 3:{}'.format(choices.count(0)/len(choices),choices.count(1)/len(choices), choices.count(2)/len(choices), choices.count(3)/len(choices)))
print(Counter(accepted_scores))

0
100
200
300
400
500
600
700
800
900
Average Score: 10154.160979999997
choice 0:0.14096718789877016 choice 1:0.36643053725763397 choice 2:0.1545003690240521 choice 3:0.3381019058195438
Counter({128: 529, 64: 341, 256: 87, 32: 40, 16: 2, 512: 1})
